* This version is higher at low z but is cleaner.

In [1]:
import subprocess
import numpy as np
import healpy as hp
import pickle
from random import randint
import joblib
import sklearn
import sys
import importlib

sys.path.append('../methods_code_Nresol/')
sys.path.append('../utils/')
from utils_circpatch import *
from methods2 import *
from vis_utils import *
import astro_cuts
importlib.reload(astro_cuts)

<module 'astro_cuts' from '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/astro_cuts.py'>

In [2]:
fullskypatch = get_testbed_dict('FullSky_Bayestar_babsgt20')

In [3]:
numtiles = np.unique(get_largepix_for_smallpix(fullskypatch['coords'], 2048, 32))

In [4]:
len(numtiles), len(numtiles)/2368

(6620, 2.795608108108108)

In [5]:
b17map = get_bayestar2017_map()
np.isnan(b17map[fullskypatch['coords']]).sum()

0

In [6]:
runname = 'fullsky_runs/16a_fwhm-15_'
wisemodel = joblib.load('spectra_matched/models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('spectra_matched/models/svm_sdss_quasarsep_unbalanced.joblib')

cuts_list = [('distmod_median_cut', {'mindm': 8}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]

outer_kwargs = {'Nsidetile': 32, 'radius_deg_extra': 35/60, 'Nsideresol': 2048,
               'stars_presaved': '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits'}
recon_kwargs = {'length': np.deg2rad(15/(2.355*60)), 'scale': 1.0, 'no_invvar': False}
kwargs_dict = {'outer_kwargs': outer_kwargs, 'recon_kwargs': recon_kwargs}
recon_func_name = 'gaussiankernel_reconstruction_effective_spherical3_blocks'

In [7]:
recondict = gnupartilewise_new(fullskypatch['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=7500, mem=8000, compile_pickle=True)

Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Ckp
Missing / Corrupted files:  [3913, 3914, 3915, 3917, 4027, 4028, 4029, 4032, 4040, 4041, 4042, 4043, 4044, 4045, 4046, 4047, 4048, 4154, 4155, 4156, 8131, 8132, 8133, 8134, 8135, 8136, 8137, 8138, 8139, 8140, 8141, 8260, 8261, 8262, 8263, 8264, 8265, 8266, 8267, 8268, 8269, 8388, 8389, 8390, 8391]


TypeError: cannot unpack non-iterable NoneType object

In [8]:
missing_tiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missing_tiles.append(tile)

In [9]:
len(missing_tiles)

45

In [10]:
missing_coords = []
for t32 in missing_tiles:
    missing_coords.append(get_smallpix_in_tilepix(outer_kwargs['Nsidetile'], t32, 2048))
missing_coords = np.hstack(missing_coords)

In [ ]:
looptilewise(missing_coords, recon_func_name, cuts_list, kwargs_dict, runname, save_tilewise=True, compile_pickle=False)

Looping over 45 tiles
Using dir  /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Region for tile3913


/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 396201 0.9921122020478226
Cut1 393961 0.9865031012818297
Cut2 391914 0.9813772846443354
Cut3 358424 0.8975162200670588
Cut4 398886 0.9988356107784881
Cut5 296874 0.7433911521443543
Cuts_All 275822 0.6906756211953895
Tile 3913, NumPixels=4096, NumStars=275822, Star/Pix ratio = 67.33935546875
SigRef 0.18033578772418446
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 642, 16%ile = 689.0, median stars = 720.0, 84%ile = 750.0, max stars = 810
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 554, 16%ile = 639.0, median stars = 672.5, 84%ile = 699.0, max stars = 745
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 593, 16%ile = 657.0, median stars = 706.0, 84%ile = 739.3, max stars = 815
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 603, 16%ile = 656.0, median stars = 683.0, 84%ile = 710.0, max stars = 765
Region for t

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 387367 0.9919921945023483
Cut1 385894 0.9882200494757922
Cut2 383573 0.9822762961786865
Cut3 354913 0.9088820826952527
Cut4 390238 0.9993444201447398
Cut5 288632 0.7391457999354664
Cuts_All 270797 0.693472883066065
Tile 3914, NumPixels=4096, NumStars=270797, Star/Pix ratio = 66.112548828125
SigRef 0.17944658056804638
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 601, 16%ile = 679.0, median stars = 712.5, 84%ile = 752.0, max stars = 805
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 530, 16%ile = 618.0, median stars = 668.5, 84%ile = 726.0, max stars = 796
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 515, 16%ile = 665.0, median stars = 703.0, 84%ile = 733.0, max stars = 778
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 558, 16%ile = 616.0, median stars = 650.0, 84%ile = 686.0, max stars = 734
Region for t

...

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 429335 0.9942130540902291
Cut1 426218 0.9869950027093745
Cut2 424442 0.9828823112584929
Cut3 383395 0.8878295826637087
Cut4 431201 0.9985341589592297
Cut5 335107 0.7760088367289283
Cuts_All 307042 0.7110185858454869
Tile 8139, NumPixels=4096, NumStars=307042, Star/Pix ratio = 74.96142578125
SigRef 0.1791054914029212
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 558, 16%ile = 679.0, median stars = 725.0, 84%ile = 759.0, max stars = 820
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 712, 16%ile = 764.0, median stars = 793.0, 84%ile = 823.0, max stars = 921
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 447, 16%ile = 669.0, median stars = 731.0, 84%ile = 781.0, max stars = 888
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 485, 16%ile = 770.0, median stars = 838.0, 84%ile = 899.3, max stars = 986
Region for ti

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 411038 0.9940363333849249
Cut1 408429 0.9877268418201517
Cut2 406312 0.9826071815508435
Cut3 365009 0.8827218116390652
Cut4 413504 1.0
Cut5 317837 0.7686431086519114
Cuts_All 292524 0.7074272558427488
Tile 8140, NumPixels=4096, NumStars=292524, Star/Pix ratio = 71.4169921875
SigRef 0.17968893763156055
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 334, 16%ile = 656.0, median stars = 719.0, 84%ile = 785.0, max stars = 874
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 280, 16%ile = 691.0, median stars = 758.0, 84%ile = 813.3, max stars = 865
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 632, 16%ile = 699.0, median stars = 758.5, 84%ile = 815.3, max stars = 890
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 581, 16%ile = 725.0, median stars = 782.0, 84%ile = 839.3, max stars = 993
Region for tile8141
Cut0 366

SigRef 0.179076633943537
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 397, 16%ile = 546.0, median stars = 582.5, 84%ile = 620.0, max stars = 677
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 352, 16%ile = 556.0, median stars = 625.0, 84%ile = 673.0, max stars = 799
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 388, 16%ile = 510.0, median stars = 612.0, 84%ile = 723.0, max stars = 811
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 579, 16%ile = 711.0, median stars = 761.0, 84%ile = 789.0, max stars = 846
Region for tile8267


In [ ]:
reconmap, varmap, reconpix = get_Nsideresol_healpix_map_from_gnuparpatches_safe(runname+'_tmpdir/', Nsideresol=kwargs_dict['outer_kwargs']['Nsideresol'], tiles = numtiles)
result_dict = {'dustmap': reconmap, 'variancemap': varmap, 'reconpix': reconpix,
               'func': 'gnupartilewise_new',
               'cuts_list': cuts_list, 'recon_func_name': recon_func_name,
               'kwargs_dict': kwargs_dict}
pickle.dump(result_dict, open(runname + '_babsgt20.pkl', 'wb'))